In [129]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import scipy.sparse as sp
from scipy.sparse.linalg import svds

from scipy.linalg import lstsq
import matplotlib.pyplot as plt

In [74]:
df = pd.read_csv('ratings.csv', nrows = 100000)

EDA

In [75]:
df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [76]:
df.shape

(100000, 4)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   userId          100000 non-null  int64  
 1   movieId         100000 non-null  int64  
 2   rating          100000 non-null  float64
 3   timestamp       100000 non-null  int64  
 4   movieId_scaled  100000 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 3.8 MB


In [97]:
# Отмасштабирую значения в колонке "movieId" для удобства работы

In [98]:
input_list = df["movieId"].unique()

In [99]:
def scale_movie_id(input_id):
    return np.where(input_list == input_id)[0][0]+1

df["movieId_scaled"] = df["movieId"].apply(scale_movie_id)

In [100]:
df.head()

,userId,movieId,rating,timestamp,movieId_scaled
0,1,110,1.0,1425941529,1
1,1,147,4.5,1425942435,2
2,1,858,5.0,1425941523,3
3,1,1221,5.0,1425941546,4
4,1,1246,5.0,1425941556,5


In [102]:
# Количество пользователей и фильмов:
n_users = df["userId"].unique().shape[0]
n_movies = df["movieId"].unique().shape[0]
print("Number_of_users: {}, Number_of_movies: {}".format(n_users, n_movies))

Number_of_users: 1014, Number_of_movies: 8832


In [103]:
# Разобьем ДС на тренировочный и тестовый
train_data, test_data = train_test_split(df, test_size = 0.2)

In [104]:
train_data.shape

(80000, 5)

In [105]:
test_data.shape

(20000, 5)

In [106]:
#Создаю две user-item матрицы для обучения и для теста
train_data_matrix = np.zeros((n_users, n_movies))
test_data_matrix = np.zeros((n_users, n_movies))

In [113]:

for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[5] - 1] = line[3]


for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[5] - 1] = line[3]    

In [114]:
print(train_data_matrix[:5, :])

[[1.  4.5 5.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  5.  ... 0.  0.  0. ]]


In [116]:
print(test_data_matrix[:5, :])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [117]:
print(train_data_matrix.shape)
print(test_data_matrix.shape)

(1014, 8832)
(1014, 8832)


In [118]:
#Использую SVD (singular value decomposition). 
# Т.О. я разложив матрицу на компоненты вновье ее перемножаю и получаю "восстановленную" матрицу с заполненными пропусками

In [120]:
#делаю SVD
u, s, vt = svds(train_data_matrix, k=10)
s_diag_matrix = np.diag(s)

In [125]:
#матрица предсказаний
x_pred = np.dot(np.dot(u, s_diag_matrix), vt)

In [126]:
print(x_pred[0])

[ 0.26794738  0.01963827  0.70504629 ...  0.         -0.00266238
  0.        ]


In [127]:
train_data_matrix[0]

array([1. , 4.5, 5. , ..., 0. , 0. , 0. ])